In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import pathlib
from pathlib import Path
import torch
sns.set_style('white')
from skimage.io import imread, imsave
import scipy.stats
import edt
from skimage import segmentation
%matplotlib qt5

In [3]:
MASKS_DIR = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/')
PHASE_DIR = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus/')
VENUS_DIR = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/venus/')

In [4]:
mask_filenames = sorted(MASKS_DIR.glob('*.png'))
phase_filenames = sorted(PHASE_DIR.glob('*.tiff'))
venus_filenames = sorted(VENUS_DIR.glob('*.tiff'))

In [5]:
index = 11
phase_img = imread(phase_filenames[index])
mask_img = imread(mask_filenames[index])
venus_img = imread(venus_filenames[index])

In [6]:
from cellbgnet.param_estimation import *
from cellbgnet.plotting import *

In [7]:
chromo_mean_var_bg_outside(venus_img, mask_img, plot=True, roi=[170, 420, 720, 720]) # central region

(111.1859724792001, 73.18108428921754)

In [8]:
73.18108428921754/ 111.1859724792001 # beta

0.6581863040583447

In [9]:
111.1859724792001 / 0.6581863040583447 # alpha

168.92781237414516

In [10]:
np.sqrt(73.18108428921754)

8.554594338086263

In [11]:
chromo_mean_var_bg_inside(venus_img, mask_img, plot=True, roi=[170, 420, 720, 720])

Max bg value: 131, max dot vale cell: 217
Mean of gamma dist bg: 117.14658314203078, variance: 72.98178795861557


(117.14658314203078, 72.98178795861557)

##### Plot to see overlay

In [12]:
overlay_cell_boundray_on_fluor(venus_img, mask_img)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [13]:
index = 21
phase_img = imread(phase_filenames[index])
mask_img = imread(mask_filenames[index])
venus_img = imread(venus_filenames[index])
edt_data = chromo_edt_mean_variance_inside(venus_img, mask_img, plot=False)

In [14]:
edt_data

{'mean': {1: 114.71998767144399,
  2: 115.84658069971462,
  3: 116.85180913939743,
  4: 117.31775407779172,
  5: 117.63915177006659,
  6: 118.81865585168019,
  7: 121.0},
 'stddev': {1: 8.340170266106986,
  2: 8.259689610017118,
  3: 8.149661957641483,
  4: 8.140308474343252,
  5: 8.04866042996527,
  6: 7.799494861438971,
  7: 6.442049363362563},
 'counts': {1: 25956, 2: 18922, 3: 13874, 4: 12752, 5: 11412, 6: 3452, 7: 12},
 'values': {1: array([124, 113, 124, ...,  99,  94, 108], dtype=uint16),
  2: array([121, 107, 112, ..., 111, 107, 101], dtype=uint16),
  3: array([113, 126, 115, ..., 106, 100, 111], dtype=uint16),
  4: array([116, 118, 110, ..., 101, 103,  96], dtype=uint16),
  5: array([115, 124, 121, ..., 108, 102, 105], dtype=uint16),
  6: array([106, 111, 111, ..., 120, 112, 114], dtype=uint16),
  7: array([125, 115, 129, 118, 113, 125, 128, 129, 118, 116, 110, 126],
        dtype=uint16)},
 'fits': {1: {'alpha': 189.9190395031804,
   'loc': -1.5,
   'beta': 0.6119449001820472

In [30]:
edt_index = 4

In [31]:
edt_data['values'][edt_index]

array([116, 118, 110, ..., 101, 103,  96], dtype=uint16)

In [32]:
low, high = edt_data['values'][edt_index].min(), edt_data['values'][edt_index].max()

In [33]:
fit_alpha_edt, fit_loc_edt, fit_beta_edt = scipy.stats.gamma.fit(edt_data['values'][edt_index], floc=-1.5)

In [34]:
low, high = edt_data['values'][edt_index].min(), edt_data['values'][edt_index].max()
fig, ax = plt.subplots()
ax.hist(edt_data['values'][edt_index], bins=np.arange(low, high), histtype='barstacked', label='data')
ax.hist(np.random.gamma(shape=fit_alpha_edt, scale=fit_beta_edt, size=len(edt_data['values'][edt_index])),
        bins=np.arange(low, high), histtype='step', label='fit')
plt.legend()
plt.show()

In [36]:
low, high

(29, 130)

In [21]:
np.mean(edt_data['values'][edt_index]), fit_alpha_edt * fit_beta_edt

(121.0, 122.5)

In [22]:
np.var(edt_data['values'][edt_index]), fit_alpha_edt * fit_beta_edt * fit_beta_edt

(41.5, 41.85368679652371)

### Get average background inside the cells

In [23]:
def get_mean_bg_inside(fluor_img, cellseg_mask, bg_cutoff_percentile=75, plot=False, dilate=True):
    if dilate:
        cell_seg_mask = segmentation.expand_labels(cellseg_mask, distance=1)
    binary_mask = cellseg_mask > 0
    inside_inds = np.where(binary_mask == 1)
    img_inside_cells = np.zeros_like(fluor_img)
    img_inside_cells[inside_inds[0], inside_inds[1]] = fluor_img[inside_inds[0], inside_inds[1]]
    
    only_pixels_inside = img_inside_cells[inside_inds[0], inside_inds[1]].ravel()
    
    # remove outliers in the inside pixels at 99%
    removing_outliers = only_pixels_inside[np.where(only_pixels_inside < np.percentile(only_pixels_inside, 99))]
    # collect only background using percentile cutoff
    collect_bg_only = removing_outliers[np.where(removing_outliers < np.percentile(removing_outliers, bg_cutoff_percentile))]
    #collect_bg_only = removing_outliers
    collect_dots_only = removing_outliers[np.where(removing_outliers > np.percentile(removing_outliers, bg_cutoff_percentile))]
    # now fit a gamma distribution and return the mean of the gamma distribution
    #print("Max value inside: ", max_values_inside)
    fit_alpha_bg, fit_loc_bg, fit_beta_bg = scipy.stats.gamma.fit(collect_bg_only, floc=0)
    fit_alpha_dots, fit_loc_dots, fit_beta_dots = scipy.stats.gamma.fit(collect_dots_only, floc=0)
    low_bg,  high_bg = collect_bg_only.min(), collect_bg_only.max()
    low_dots, high_dots = collect_dots_only.min(), collect_dots_only.max()
    if plot:
        fig, ax = plt.subplots(nrows=1, ncols=2)
        ax[0].hist(collect_bg_only, bins=np.linspace(low_bg, high_bg), histtype='barstacked', label='data')
        ax[0].hist(np.random.gamma(shape=fit_alpha_bg, scale=fit_beta_bg, size=len(collect_bg_only)+0),
                        bins=np.linspace(low_bg, high_bg), histtype='step', label='fit')
        ax[0].legend()
        ax[0].set_title('Bg only')
        ax[1].hist(collect_dots_only, bins=np.linspace(low_dots, high_dots), histtype='step', label='data')
        ax[1].hist(np.random.gamma(shape=fit_alpha_dots, scale=fit_beta_dots, size=len(collect_dots_only)+0),
                        bins=np.linspace(low_dots,  high_dots), histtype='step', label='fit')
        ax[1].set_title('Dots only')
        ax[1].legend()
        plt.show()
    
    print(f"Max bg value: {collect_bg_only.max()}, max dot vale cell: {collect_dots_only.max()}")
    print(f"mean of gamma dist bg: {fit_alpha_bg * fit_beta_bg}, variance: {fit_alpha_bg * fit_beta_bg * fit_beta_bg}")
    return fit_alpha_bg * fit_beta_bg

In [24]:
index = 10
phase_img = imread(phase_filenames[index])
mask_img = imread(mask_filenames[index])
venus_img = imread(venus_filenames[index])

In [25]:
get_mean_bg_inside(venus_img, mask_img, plot=True)

Max bg value: 134, max dot vale cell: 231
mean of gamma dist bg: 118.80710526629832, variance: 82.91038367990615


118.80710526629832

In [26]:
(118.8071 - 103.0) * 0.39 / 0.95

6.489230526315792

#### Get average outside the cells

In [27]:
def get_mean_bg_outside(fluor_img, cellseg_mask, dilate=True, roi=[], plot=False):
    """
    Arguments:
    -----------
        fluor_img:
        cell_seg_mask:
        dilate:
        roi: []
    """
    if dilate:
        cell_seg_mask = segmentation.expand_labels(cellseg_mask, distance=1)
    binary_mask = cellseg_mask == 0
    
    img_outside_cells = np.zeros_like(fluor_img)
    outside_inds = np.where(binary_mask == 1)
    img_outside_cells[outside_inds[0], outside_inds[1]] = fluor_img[outside_inds[0], outside_inds[1]]
    
    only_pixels_outside = img_outside_cells[outside_inds[0], outside_inds[1]].ravel()
    
    # remove outliers in the inside pixels at 99%
    collect_bg_only = only_pixels_outside[np.where(only_pixels_outside < np.percentile(only_pixels_outside, 99))]
    # now fit a gamma distribution and return the mean of the gamma distribution
    fit_alpha, fit_loc, fit_beta = scipy.stats.gamma.fit(collect_bg_only, floc=0)
    low, high = collect_bg_only.min(), collect_bg_only.max()
    if plot:
        fig, ax = plt.subplots()
        ax.hist(collect_bg_only, bins=np.linspace(low, high), histtype='step', label='data')
        ax.hist(np.random.gamma(shape=fit_alpha, scale=fit_beta, size=len(collect_bg_only)+0),
                        bins=np.linspace(low, high), histtype='step', label='fit')
        plt.legend()
        plt.show()
    
    # returning the mean and variance of the fitted image
    return fit_alpha * fit_beta, fit_alpha * fit_beta * fit_beta
    

In [28]:
mask_img = imread(mask_filenames[0])
venus_img = imread(venus_filenames[0])

In [29]:
get_mean_bg_outside(venus_img, mask_img, plot=True)

(110.00883193476756, 89.31424145180127)

In [30]:
mean_outside = []
var_outside = []
for index in range(len(phase_filenames)):
    mask_img = imread(mask_filenames[index])[170: 170 + 720, 420: 420 + 720]
    venus_img = imread(venus_filenames[index])[170: 170 + 720, 420: 420 + 720]
    one_img_mean, one_img_var = get_mean_bg_outside(venus_img, mask_img)
    mean_outside.append(one_img_mean)
    var_outside.append(one_img_var)

KeyboardInterrupt: 

In [ ]:
np.mean(mean_outside)

In [ ]:
np.mean(var_outside)

### Cell bg as function of edt from the center line of the cell

In [ ]:
index = 140
cellseg_mask = imread(mask_filenames[index])
fluor_img = imread(venus_filenames[index])
bg_cutoff_percentile = 75
cellseg_mask = segmentation.expand_labels(cellseg_mask, distance=1)

binary_mask = cellseg_mask > 0
inside_inds = np.where(binary_mask == 1)
img_inside_cells = np.zeros_like(fluor_img)
img_inside_cells[inside_inds[0], inside_inds[1]] = fluor_img[inside_inds[0], inside_inds[1]]

only_pixels_inside = img_inside_cells[inside_inds[0], inside_inds[1]].ravel()

# remove outliers in the inside pixels at 99%
removing_outliers = only_pixels_inside[np.where(only_pixels_inside < np.percentile(only_pixels_inside, 99))]
# collect only background using percentile cutoff
collect_bg_only = removing_outliers[np.where(removing_outliers < np.percentile(removing_outliers, bg_cutoff_percentile))]
#collect_bg_only = removing_outliers
collect_dots_only = removing_outliers[np.where(removing_outliers > np.percentile(removing_outliers, bg_cutoff_percentile))]
# now fit a gamma distribution and return the mean of the gamma distribution
#print("Max value inside: ", max_values_inside)


In [ ]:
cutoff_bg = np.percentile(removing_outliers, bg_cutoff_percentile)

In [ ]:
cutoff_bg

In [ ]:
just_bg = img_inside_cells.copy()
just_bg[just_bg > cutoff_bg] = 0.0

In [ ]:
plt.figure()
plt.imshow(just_bg)
plt.show()

In [ ]:
plt.figure()
plt.imshow(img_inside_cells)
plt.show()

In [ ]:
dists = edt.edt(cellseg_mask)

In [ ]:
dists

In [ ]:
plt.figure()
plt.imshow(dists)
plt.show()

In [ ]:
np.unique(dists, return_counts=True)

In [ ]:
dists.min(), dists.max()

In [ ]:
edt_1 = just_bg[dists==1]
edt_1 = edt_1[edt_1 > 0]
edt_2 = just_bg[dists==2]
edt_2 = edt_2[edt_2 > 0]

edt_3 = just_bg[dists==3]
edt_3 = edt_3[edt_3 > 0]

edt_4 = just_bg[dists==4]
edt_4 = edt_4[edt_4 > 0]

edt_5 = just_bg[dists==5]
edt_5 = edt_5[edt_5 > 0]

edt_6 = just_bg[dists==6]
edt_6 = edt_6[edt_6 > 0]


In [ ]:
def plot_dist(array):
    low, high = array.min(), array.max()
    fig, ax = plt.subplots()
    ax.hist(array, bins=np.linspace(low, high), histtype='bar', label='data')
    plt.show()

In [ ]:
plot_dist(edt_5)

In [ ]:
np.mean(edt_1), np.mean(edt_2), np.mean(edt_3), np.mean(edt_4), np.mean(edt_5), np.mean(edt_6)

In [ ]:
np.std(edt_1), np.std(edt_2), np.std(edt_3), np.std(edt_4), np.std(edt_5), np.std(edt_6)

In [ ]:
(126 - 103) * 0.39

In [ ]:
def get_cell_bg_pattern(fluor_img, cellseg_mask, dilate=True, bg_percentile_cutoff=75, plot=False):
    """
    Arguments:
    ----------
        fluor_img: 
        cellseg_mask:
        dilate:
        bg_percentile_cutoff
        plot:
    """
    